# Análisis del Sentimiento de las noticias económicas en Chile 2020

En esta sección se desarrolla el **modelo clasificador de sentimiento económico**. 

El modelo se entrenó con una base de datos de textos económicos, constituida por noticias económicas publicadas en diversos medios de prensa online en Chile, durante abril de 2020 y abril de 2021. Las noticias fueron etiquetadas manualmente como pesimistas, neutrales y optimistas.  Adicionalmente, la base de datos contiene fragmentos de cartas a los accionistas de años anteriores, en que claramente se observa una polaridad  específica: pesimista, neutral u optimista. 

Los textos debían cumplir la condición de expresar claramente ideas con cierta polaridad, sin ambigüedades que pudieran confundir al algoritmo. Si, por ejemplo, un texto contenía una mezcla de ideas pesimistas y optimistas, se optaba por omitirlo, o bien se separaban los textos fragmentos en con etiquetas individuales. Otra condición importante fue que los textos contuvieran información económica, de forma que las palabras y combinaciones de palabras encontradas coincidieran con las que pudiéramos encontrar en las cartas a los accionistas. En otras palabras, los textos debían contener la jerga mayormente utilizada en los ámbitos de la economía, negocios, finanzas, marketing, etc.


In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import unidecode

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

En primer lugar se cargan los datos. Luego:
- Se reemplaza la etiqueta de sentimiento: 2 (neutral), por 0 (no positivo). El clasificador se entrenará sólo para identificar el sentimiento positivo.
- Se eliminaron los *missing values* (NA).
- Se seleccionó sólo un conjunto de las variables de la base de datos: (1) response: dummy igual 1 para los textos con sentimiento positivo, y 0 en otro caso; y (2) texto: noticia o fragmento de carta a los accionistas con una polaridad específica.


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/percepcioneseconomicas/publicaciones/main/sentiment_data/sentiment_data.csv')
data['response'][data['response'] == 2.0] = 0
data = data.dropna().reset_index(drop=True)
data = data[['response', 'texto']]
data.head()

,response,texto
0,1.0,"En cuanto a las colocaciones, estas alcanzaron..."
1,0.0,En el año 2009 el mundo enfrentó una crisis fi...
2,1.0,Es para mí motivo de gran satisfacción compart...
3,1.0,Si tuviera que resumir la gestión 2012 de Cruz...
4,1.0,El crecimiento anteriormente mencionado redund...


La base de datos contiene **1461 observaciones** (textos económicos clasificados según polaridad).

In [3]:
print('Dimensiones de los datos: \n', data.shape, '\n')
print(data.info())

Dimensiones de los datos: 
 (1461, 2) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   response  1461 non-null   float64
 1   texto     1461 non-null   object 
dtypes: float64(1), object(1)
memory usage: 23.0+ KB
None


En la siguiente celda se observa que el 77% de los textos tienen un sentimiento negativo (0), mientras que sólo el 23% tiene un sentimiento positivo. Esto se debe a que los textos son principalmente noticias económicas, publicadas durante la pandemia de covid19 (año 2020), cuando la actividad económica estaba siendo seriamente afectada por la contingencia.

In [4]:
# Rows en cada categoría de la variable y
data['response'].value_counts(normalize=True).round(2)

0.0    0.77
1.0    0.23
Name: response, dtype: float64

El siguiente análisis muestra el número de palabras de la noticia más larga (2403 palabras) y la más corta (37 palabras). Se aprecia una gran diferencia en la extensión de los textos.

In [5]:
# Número de palabras
x = []
[x.append(len(e.split())) for e in data['texto']]
print('Número de palabras de la noticia más larga: \n',  max(x), '\n')
print('Número de palabras de la noticia más corta: \n',  min(x), '\n')

Número de palabras de la noticia más larga: 
 2403 

Número de palabras de la noticia más corta: 
 37 



En la siguiente celda se muestra un texto de la base de datos seleccionado aleatoriamente.

In [13]:
# Noticia aleatoria
k = np.random.randint(0, len(data['texto']))
print('Noticia %d:' % k, '\n', data['texto'][k])

Noticia 637: 
 McDonald’s informó este martes una pronunciada caída en sus beneficios durante el segundo trimestre, debido a la baja de ventas producto de los cierres por la pandemia.

La cadena de comida rápida sufrió un descenso del 68% en sus ganancias a 483,8 millones de dólares, tras una baja del 30% en sus ingresos, hasta los 3.800 millones.

Las ventas se hundieron en los principales mercados en los que McDonald’s está presente.

Sin embargo, la cadena sostuvo que en Estados Unidos no fue tan grave gracias a que su servicio de comida para llevar continuó, pese al cierre de locales al público.


# Preprocesamiento de los datos

En esta sección se preprocesan los datos para su posterior utilización.

En la siguiente celda se carga una lista de **Stopwords**. Las StopWords son palabras sin un significado o sentido claro—como los artículos, pronombres y preposiciones—que se usan con frecuencia en todo tipo de textos, independientemente de la polaridad, por lo que no contribuyen a la clasificación de las cartas.

In [14]:
# Lista de stopwords
sw = pd.read_csv('spanish.txt', header=None, names=['stopwords'])
stopwords = sw['stopwords'].tolist()

Se define una función para preprocesar los textos. Esta función:
- Pone todas las palabras en minúsculas.
- Quita números y caracteres especiales.
- Separa los textos en palabras individuales.
- Mantiene sólo las palabras que no sean Stopwords (remueve las Stopwords)
- Une las palabras nuevamente para formar un texto.

In [15]:
# Función para preprocesar los datos.
def preprocess(s):
    s = s.lower()
    s = re.sub('[0-9]+', '', s) 
    s = re.sub('[!"#$%&()*+,-./:;<=>¿?@[\\]^_`{|}~\t—’‘“”]', '', s)
    tokens = nltk.tokenize.word_tokenize(s) 
    tokens = [t for t in tokens if t not in stopwords] 
    tokens = [unidecode.unidecode(t) for t in tokens]
    jtokens = ' '.join(tokens)
    return jtokens

En la siguiente celda se aplica la función anterior a los textos de la base de datos.

In [16]:
# Preprocesamiento
pdata = [preprocess(t) for t in data['texto']]

Luego, se muestra la misma noticia de antes, después del pre procesamiento.

In [17]:
print('Noticia %d :' % k, '\n', pdata[k])

Noticia 637 : 
 mcdonalds informo martes pronunciada caida beneficios segundo trimestre debido baja ventas producto cierres pandemia cadena comida rapida sufrio descenso ganancias millones dolares tras baja ingresos millones ventas hundieron principales mercados mcdonalds presente embargo cadena sostuvo unidos tan grave gracias servicio comida llevar continuo pese cierre locales publico


## Vectorización 

A continuación, se vectorizan los datos. Vectorizar significa dar una estructura de dataframe (matricial) a los datos, donde las cartas son las filas, y el *vocabulario* son las columnas. 
- El *vocabulario* es una lista de todas las palabras presentes en el total de textos. Este vocabulario se compone de las 3000 palabras más frecuentes.

Los valores de la matriz pueden ser:
- Recuento de palabras por texto, es decir, el número de veces que aparece cada palabra en el texto correspondiente.
- Frecuencia porcentual de palabras por texto, donde la cifra anterior se divide por el total de palabras en el texto correspondiente, de forma de normalizar por la extensión de las cartas.
- **Tf-idf** (term frequency – Inverse document frequency), métrica que multiplica la frecuencia con que cada palabra aparece por el inverso de la frecuencia con que aparece en el total de textos, lo que hace que las palabras más comunes tengan un menor peso en la matriz.

Además de las palabras individuales se consideraron **n-grams**, que son combinaciones de palabras consecutivas. 

Esto es importante, ya que una palabra como *desempleo* podría tener una connotación negativa, pero su sentido cambia a positivo si se combina con otra palabra. Por ejemplo, si encontramos la combinación *desempleo disminuye*. 

De esta manera, se procesaron los datos de **seis** formas distintas:

1. Recuento de palabras individuales.
2. Recuento de ngrams 1 y 2.
3. Porcentaje de palabras individuales.
4. Porcentaje de ngrams 1 y 2.
5. Tf-idf de palabras individuales.
6. Tf-idf de ngrams 1 y 2.



In [18]:
# Conteo de palabras por texto
vect = CountVectorizer(max_features=3000)
vdat = vect.fit_transform(pdata)
data1 = pd.DataFrame(vdat.toarray(), columns=vect.get_feature_names())
data1.head(1)

,aa,abastecimiento,abiertas,abogado,abril,abriljunio,abrio,abrir,aca,academico,...,web,wti,xtb,yapocl,york,yuanes,zaldivar,zona,zonas,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Conteo de palabras y ngrams por texto
vect = CountVectorizer(max_features=3000, ngram_range=(1,2))
vdat = vect.fit_transform(pdata)
data2 = pd.DataFrame(vdat.toarray(), columns=vect.get_feature_names())
data2.head(1)

,abastecimiento,abogado,abril,abril junio,abril mayo,abriljunio,abrir,aca,academico,acceder,...,walmart,washington,web,wti,yapocl,york,zaldivar,zona,zona euro,zonas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Frecuencia de palabras por texto
data1sum = data1.sum(axis=1)
data3 = data1.divide(data1sum, axis=0)
data3.head(1)

,aa,abastecimiento,abiertas,abogado,abril,abriljunio,abrio,abrir,aca,academico,...,web,wti,xtb,yapocl,york,yuanes,zaldivar,zona,zonas,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Frecuencia de palabras y ngrams por texto
data2sum = data2.sum(axis=1)
data4 = data2.divide(data2sum, axis=0)
data4.head(1)

,abastecimiento,abogado,abril,abril junio,abril mayo,abriljunio,abrir,aca,academico,acceder,...,walmart,washington,web,wti,yapocl,york,zaldivar,zona,zona euro,zonas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Tfidf de palabras por texto
vect = TfidfVectorizer(max_features=3000)
vdat = vect.fit_transform(pdata)
data5 = pd.DataFrame(vdat.toarray(), columns=vect.get_feature_names())
data5.head(1)

,aa,abastecimiento,abiertas,abogado,abril,abriljunio,abrio,abrir,aca,academico,...,web,wti,xtb,yapocl,york,yuanes,zaldivar,zona,zonas,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Tfidf de palabras por texto
vect = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
vdat = vect.fit_transform(pdata)
data6 = pd.DataFrame(vdat.toarray(), columns=vect.get_feature_names())
data6.head(1)

,abastecimiento,abogado,abril,abril junio,abril mayo,abriljunio,abrir,aca,academico,acceder,...,walmart,washington,web,wti,yapocl,york,zaldivar,zona,zona euro,zonas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Separación de la muestra

A continuación se separa la muestra en tres subconjuntos de datos, seleccionados de forma aleatoria y estratificada, de forma tal que la distribución de la variable dependiente sea la misma en cada subset. Los subsets tienen 949, 409 y 103 textos cada uno, es decir, 65%, 28% y 7% de los datos, respectivamente. 

Esta separación de la muestra se hace para cada una de las seis matrices de datos. Por eso en la celda final se guardan los índices de los distintos subconjuntos de los datos, para más tarde usarlos para seleccionar las observaciones de cada una de las seis bases de datos procesadas anteriormente.

Lógica del modelamiento:
- El **subset 1** se usa para entrenar los modelos individuales, seis regresiones logísticas, una para cada matriz de datos, donde cada una contiene los textos procesados de una forma diferente. 
- Luego, cada uno de los modelos generó predicciones para los subsets 2 y 3. 
- A continuación, se entrenó el **metamodelo** usando el **subset 2**, es decir, usando las predicciones de los primeros seis modelos para el subset 2 como inputs (como regresores o predictores), y usando la variable dependiente del subset 2. 
- Finalmente, se evaluó el desempeño de los modelos en el subset 3. 

Esta estrategia permite **reducir el sobreajuste** de los modelos a los datos.


In [24]:
# Se define la variable y se toman muestras
y = data['response']
X = data1

In [25]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.35, stratify=y, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.2, stratify=y_val, random_state=123)

In [26]:
samples = np.array([y_train.shape[0], y_val.shape[0], y_test.shape[0]])
print(samples)
print((samples/sum(samples)*100).round())


[949 409 103]
[65. 28.  7.]


In [27]:
i_train = y_train.index
i_val = y_val.index
i_test = y_test.index

# Funciones y dataframes útiles

A continuación se definen algunas funciones útiles para el posterior modelamiento.

La primera función calcula métricas para la evaluación del desempeño de los modelos.

Estas métricas son:
- **CV-Score**: accuracy, obtenida por cross-validation (se trata de una estimación de la accuracy fuera de la muestra). **Accuracy** es una medida que muestra la suma de predicciones correctas (POSITIVOS y NEGATIVOS) sobre el total de predicciones.
- **Accuracy**: obtenida comparando los valores reales con los predichos por el modelo. 
- **AUC**: área bajo la curva ROC. Se interpreta como la probabilidad de que un algoritmo clasificador asigne una mayor probabilidad a una observación clasificada como positiva, que fue seleccionada aleatoriamente, que a una observación clasificada como negativa seleccionada aleatoriamente. Los valores que puede tomar esta área varían entre 0.5 y 1, donde 1 representa un clasificador perfecto, y 0.5 es un clasificador sin capacidad discriminatoria. 
- **F1 Score**: media harmónica de dos métricas: Precision & Recall. Toma su mejor valor en 1 y su peor valor en 0. 


- **Precision** muestra qué porcentaje de las observaciones clasificadas como positivas por el modelo son efectivamente positivas (un modelo con alta precisión es un modelo con pocos falsos positivos). 
- **Recall** (sensitivity) muestra qué porcentaje de las observaciones efectivamente positivas, fueron clasificadas como positivas por el modelo (un modelo con alto recall es un modelo con pocos falsos negativos, es decir, un modelo que clasifica correctamente la mayoría de los casos efectivamente positivos).

In [28]:
# Función para calcular las métricas de evaluación. 
def get_metrics(modelo, y, y_pred, y_pred_proba):
    return pd.DataFrame({
                'CV-Score': searcher.best_score_,
                'Accuracy': accuracy_score(y, y_pred),
                'AUC': roc_auc_score(y, y_pred_proba),
                'F1 Score': f1_score(y, y_pred)},
                index=[modelo])

La siguiente función toma como input una matriz, y entrega tres subconjuntos de datos. 
- El input es cualquiera de las seis bases de datos cuyos textos fueron procesados de forma diferente.
- El output es un subconjunto de datos seleccionados aleatoriamente, para entrenar los modelos individuales y el metamodelo (subsets 1, 2 y 3).

In [29]:
def gen_X_train(X):
    X_train = X.loc[i_train]
    X_val = X.loc[i_val]
    X_test = X.loc[i_test]
    return X_train, X_val, X_test

La siguiente función guarda las predicciones de cada modelo.

In [30]:
def save_preds(pred_val, pred_test, modelo, y_pred_proba_val, y_pred_proba_test):
    pred_val[modelo] = y_pred_proba_val
    pred_test[modelo] = y_pred_proba_test

También se crean algunos **dataframes vacíos** para almacenar los resultados (métricas de evaluación) y parámetros óptimos de los modelos.

In [32]:
# DataFrames vacíos para almacenar los resultados 
results = pd.DataFrame()
parametros = pd.DataFrame()

Por último, se crea un diccionario, para almacenar las seis bases de datos, donde en cada una se procesaron los valores, los pesos de las palabras, de forma diferente.

In [33]:
# Diccionario con los datos y sus nombres
keys = ['data1', 'data2', 'data3', 'data4', 'data5', 'data6']
values = [data1, data2, data3, data4, data5, data6]
datos = dict(zip(keys, values))

# Modelos primera ronda

A continuación se seleccionan los modelos de la primera ronda, es decir, los modelos individuales: las seis regresiones logísticas entrenadas, cada una, con una base de datos distinta. Se optó por la regresión logística (**logit**), ya que se trata de un método más eficiente para el tipo de datos con que se cuenta, donde muchos valores son iguales a cero, y otros toman valores discretos.

Para **calibrar** los modelos se usó **5-fold cross-validation** y una **búsqueda de grilla**.

La búsqueda de grilla implica ajustar cada modelo utilizando un conjunto de valores para los parámetros exógenos, que en el caso de la **regresión logística** pueden ser: 

1. tipo de regularización, Lasso o Ridge.
2. el parámetro asociado a la regularización (lambda).  

Se probaron 20 posibilidades para el valor lambda, que junto con los dos posibles métodos de regularización significó probar 40 combinaciones para cada uno de los seis modelos individuales. De esta manera se seleccionó la combinación óptima de parámetros, que es aquella que minimiza el CV-Error (CV-Score).

In [34]:
model = LogisticRegression(random_state=123)
parameters = {'C':np.logspace(-4, 4, 20), 
               'penalty':['l1', 'l2']}
searcher = GridSearchCV(estimator=model, 
                        param_grid=parameters, 
                        scoring='f1',
                        n_jobs=-1, 
                        verbose=1)

for key, value in datos.items():
    X_train, X_val, X_test = gen_X_train(value)
    
    searcher.fit(X_train, y_train)
    
    print(key, "Best CV params", searcher.best_params_)
    parametros = parametros.append(pd.DataFrame(searcher.best_params_, index=[key]))

    best_model = searcher.best_estimator_
    y_pred_train = best_model.predict(X_train)
    y_pred_proba_train = best_model.predict_proba(X_train)[:,1]
    results = results.append(get_metrics(key, y_train, y_pred_train, y_pred_proba_train))

Fitting 5 folds for each of 40 candidates, totalling 200 fits
data1 Best CV params {'C': 1.623776739188721, 'penalty': 'l2'}
Fitting 5 folds for each of 40 candidates, totalling 200 fits
data2 Best CV params {'C': 0.615848211066026, 'penalty': 'l2'}
Fitting 5 folds for each of 40 candidates, totalling 200 fits
data3 Best CV params {'C': 10000.0, 'penalty': 'l2'}
Fitting 5 folds for each of 40 candidates, totalling 200 fits
data4 Best CV params {'C': 3792.690190732246, 'penalty': 'l2'}
Fitting 5 folds for each of 40 candidates, totalling 200 fits
data5 Best CV params {'C': 10000.0, 'penalty': 'l2'}
Fitting 5 folds for each of 40 candidates, totalling 200 fits
data6 Best CV params {'C': 1438.44988828766, 'penalty': 'l2'}


# Resultados primera ronda

A continuación se muestran los resultados para la primera ronda.
- El mejor modelo según CV-Score fue el 5 (tf-idf para palabras individuales).
- Según todas las demás mérticas el mejor modelo fue el 1 (recuento de palabras individuales)

El CV-Score muestra que el f1 score se encontró cerca de 0.70 fuera de la muestra (*out-of-sample*). Las otras métricas muestran que, dentro de la muestra (*in-sample*), la clasificación fue prácticamente perfecta. La diferencia entre ambas métricas indica sobreajuste.

In [35]:
print('Best models:')
print(results.idxmax(), '\n')
results

Best models:
CV-Score    data5
Accuracy    data1
AUC         data1
F1 Score    data1
dtype: object 



,CV-Score,Accuracy,AUC,F1 Score
data1,0.702219,1.000000,1.0,1.000000
data2,0.704159,1.000000,1.0,1.000000
data3,0.683685,1.000000,1.0,1.000000
data4,0.680060,0.998946,1.0,0.997712
data5,0.704350,1.000000,1.0,1.000000
data6,0.700581,1.000000,1.0,1.000000


Por último, se guardan los mejores parámetros identificados para su posterior utilización.

In [36]:
parametros.to_csv('parametros_1.csv')
parametros

,C,penalty
data1,1.623777,l2
data2,0.615848,l2
data3,10000.000000,l2
data4,3792.690191,l2
data5,10000.000000,l2
data6,1438.449888,l2


# Ajuste de los modelos a la muestra completa

A continuación se ajustan los modelos a la muestra completa. Hay que mencionar que este paso es redundante, ya que sklearn hace esto automáticamente después de elegir el mejor modelo a través de la búsqueda de grilla. Eso no lo sabía al momento de hacer esto, pero se aprovecha la instancia para guaardar las predicciones de los modelos, que se usarán como inputs del metamodelo.

In [37]:
# DataFrames vacíos para almacenar los resultados 
pred_val = pd.DataFrame()
pred_test = pd.DataFrame()

In [38]:
for key, value in datos.items():
    X_train, X_val, X_test = gen_X_train(value)

    model = LogisticRegression(random_state=123,
                                C=parametros.loc[key][0],
                                penalty=parametros.loc[key][1])

    model.fit(X_train, y_train)
    
    y_pred_proba_val = model.predict_proba(X_val)[:,1]
    y_pred_proba_test = model.predict_proba(X_test)[:,1]
    save_preds(pred_val, pred_test, key, y_pred_proba_val, y_pred_proba_test)

In [39]:
pred_val.index = i_val
pred_test.index = i_test

# Segunda ronda: Stacking

A continuación se elige el **metamodelo**. Este es un método de ensemble (ensamblado o meta ensamblado), donde se combinan las predicciones de otros modelos, para obtener una predicción superior a la de los modelos individuales.

Se repitió la búsqueda de grilla con cross-validation para el metamodelo, de forma de seleccionar sus parámetros óptimos. El metamodelo también es una regresión logística, y se buscó en la misma grilla utilizada en los modelos individuales. Una vez elegido el mejor metamodelo, se ajustó al subset2 completo y se hizo la predicción para el subset 3. Por último, se guardaron los parámetros óptimos.

Notar que los resultados no son tan buenos como los anteriores, lo que se debe a que el metamodelo se entrenó con datos distintos, usando las predicciones (fuera de la muestra) de los modelos anteriores como inputs, no se usaron los textos originales. Esto se hace así para reducir el sobreajuste del metamodelo. Además, todas las métricas de error son 'out-of-sample', ya que se estimaron, ya sea por cross-validation, o bien usando el subset 3 (no utilizado hasta ahora). 

In [40]:
X_val = pred_val
X_test = pred_test

In [41]:
# DataFrames vacíos para almacenar los resultados 
parametros = pd.DataFrame()
results = pd.DataFrame()
pred_test = pd.DataFrame()

In [42]:
modelo = 'LR'
model = LogisticRegression(random_state=123)
parameters = {'C':np.logspace(-4, 4, 20), 
              'penalty':['l1', 'l2']}
searcher = GridSearchCV(estimator=model, 
                        param_grid=parameters, 
                        scoring='f1',
                        n_jobs=-1, 
                        verbose=1)
searcher.fit(X_val, y_val)
print("Best CV params", searcher.best_params_)
parametros = parametros.append(pd.DataFrame(searcher.best_params_, index=[key]))

best_model = searcher.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:,1]
pred_test[modelo] = y_pred_proba

results = results.append(get_metrics(modelo, y_test, y_pred, y_pred_proba))
results.round(4)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best CV params {'C': 29.763514416313132, 'penalty': 'l2'}


,CV-Score,Accuracy,AUC,F1 Score
LR,0.6731,0.8641,0.9014,0.6667


In [43]:
parametros.to_csv('parametros_2.csv')
parametros

,C,penalty
data6,29.763514,l2


# Ajuste del mejor modelo a la muestra completa

In [44]:
modelo = 'LR'
model = LogisticRegression(random_state=123,
                            C=parametros['C'].iloc[0],
                            penalty=parametros['penalty'].iloc[0])
model.fit(X_val, y_val)
y_pred_proba = model.predict_proba(X_test)[:,1]

# Optimizar threshold

El último paso es **optimizar el threshold**. 

El metamodelo entrega como predicción la probabilidad de que un texto sea optimista, que es un valor entre 0.0 y 1.0. 

Para hacer una predicción de las etiquetas 0 y 1 se necesita un threshold (un umbral) tal que una probabilidad mayor al threshold se considere como un 1 (etiqueta positiva). 

- Para optimizar el threshold se hizo una búsqueda de grilla sencilla, en 300 valores equidistantes entre 0 y 1. 
- Se optimizó usando como referencia la métrica **F1 Score**, que es la media harmónica de otras dos métricas precision (porcentaje de los datos clasificados como positivos por el modelo que son efectivamente positivos) y recall (porcentaje de los datos efectivamente positivos que fueron clasificados como positivos por el modelo). 

In [45]:
# Función para transformar probabilidad en label
def to_labels(y_pred_proba, threshold):
	return (y_pred_proba >= threshold).astype('int')

In [46]:
# Diferentes thresholds a testear
thresholds = np.linspace(0, 1, 300)

In [49]:
# Optimización del threshold en base a F1 Score
scores = [f1_score(y_test, to_labels(y_pred_proba, t)) for t in thresholds]
ix = np.argmax(scores)
print(modelo, 'Threshold=%.4f, F-Score=%.4f' % (thresholds[ix], scores[ix]))

LR Threshold=0.1873, F-Score=0.7234


In [50]:
y_pred = (y_pred_proba >= thresholds[ix]).astype(int)
y_pred = pd.DataFrame(y_pred, columns=['y_pred'], index=y_test.index)

Finalmente se calcula la accuracy del metamodelo tras optimizar el threshold. Se encuentra que clasifica correctamente el 87% de los sentimientos.

In [51]:
print('Accuracy=%.4f' % accuracy_score(y_test, y_pred).round(4))

Accuracy=0.8738


In [52]:
pred = pd.concat([y_test, y_pred], axis=1)
pred['Accuracy'] = (pred['response']==pred['y_pred'])
pred

,response,y_pred,Accuracy
531,0.0,0,True
761,0.0,0,True
848,0.0,0,True
362,0.0,0,True
696,0.0,0,True
...,...,...,...
74,0.0,0,True
1053,0.0,0,True
1201,1.0,1,True
1236,0.0,0,True
